# Part 2 — Model Quality Monitoring

<div class="alert alert-warning"> This notebook has been last tested on a SageMaker Studio JupyterLab instance using the <code>SageMaker Distribution Image 3.0.1</code> and with the SageMaker Python SDK version <code>2.245.0</code></div>

In this notebook you are going to use [Amazon SageMaker model monitor](https://aws.amazon.com/sagemaker/model-monitor/) to add continuous and automated [monitoring of the model quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html) for the traffic to your real-time SageMaker inference endpoints. You also implement [model monitoring](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html) to detect performance drift and model metric anomalies.

Using Model Monitor integration with [Amazon EventBridge](https://aws.amazon.com/eventbridge/) you can implement automated response and remediation to any detected issues with data and model quality. For example, you can launch an automated model retraining if the model performance falls below a specific threshold.

Additionally to data and model quality monitoring you can implement [bias drift](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html) and [feature attribution drift](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) monitoring.
    
##  Context

In this deployment module, you have:
1. ✅ **Pre-provisioned**: SageMaker Unified Studio domain with registered models
2. ✅ **Deployed**: SageMaker endpoint with data capture enabled - preprovsioned 
3. ✅ **Tested**: Basic endpoint functionality in the previous notebook
4. 🎯 **Now**: Set up comprehensive model monitoring

## Prerequisites
- Completed Lab 5.1: Model approved and triggered CDK for endpoint deployment
- SageMaker endpoint with data capture enabled
- **IAM Permissions**: Your execution role must have Model Monitor permissions

### ⚠️ Important: IAM Permissions Required

Your IAM role needs these additional permissions for Model Monitor:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateDataQualityJobDefinition",
                "sagemaker:CreateModelQualityJobDefinition",
                "sagemaker:CreateMonitoringSchedule",
                "sagemaker:DescribeMonitoringSchedule",
                "sagemaker:ListMonitoringSchedules",
                "sagemaker:StopMonitoringSchedule",
                "sagemaker:DeleteMonitoringSchedule",
                "sagemaker:CreateProcessingJob",
                "sagemaker:DescribeProcessingJob"
            ],
            "Resource": "*"
        }
    ]
}
```

**If you get AccessDeniedException errors:**
1. Go to AWS Console → IAM → Roles
2. Find your execution role (shown in setup section below)
3. Add the above permissions as an inline policy

---

<div class="alert alert-info"> 💡
This notebook contains scripts for :<br/>
- Part 2: Monitor Model quality<br/>

<br/>

You need approximately between 40 and 60 minutes to go through this notebook. To optimize time you can execute both parts independently. For both parts you must execute all following sections up to the <strong>Part 1</storng>.
</div>

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

## 1) Environment & configuration

This cell sets up AWS SDK clients, your SageMaker session/role, and base S3 locations we will use throughout the notebook.

**Why this matters:** Model Monitor writes/reads artifacts from S3 and needs an execution role. Keep track of:
- **Bucket**: where baselines, reports, and captured data live
- **Prefix**: root path inside the bucket


<div class="alert alert-info"> 💡
This workshop uses **Python 3** kernel in SageMaker Studio JupyterLab.  
If you see import errors, select the **Python 3 (Data Science)** kernel and **Restart Kernel & Clear Output**.
</div>


## Model Monitoring Architecture (quick refresher)

1. **Data capture** writes requests/response payloads from your real-time endpoint to S3.  
2. **Baselines** compute reference metrics/statistics from a labeled validation set.  
3. A **monitoring schedule** runs on a cadence (e.g., hourly) to evaluate new data.  
4. For **Model Quality**, the schedule **joins predictions with ground truth** and computes metrics (accuracy, precision/recall/F1, etc.).  
5. **Reports & violations** are written to S3 and surfaced in CloudWatch.


### Before you begin — parameters to verify

Update the following variables in the **Setup** section if needed:

```python
endpoint_name = "model-deploy-16-21-26-26-staging"
problem_type = "BinaryClassification"   # BinaryClassification | MulticlassClassification | Regression
inference_attribute = "prediction"
probability_attribute = "probability"  # classification only
ground_truth_attribute = "label"
bucket = "<your-bucket>"
prefix = "<your-prefix>"
```


## How Model Monitor works
Amazon SageMaker Model Monitor automatically monitors ML models in production and notifies you when quality issues arise. Model Monitor uses rules to detect drift in your models and data and alerts you when it happens. The following figure shows how this process works.

![](img/data-monitoring-architecture.png)

The process for setting up and using the data monitoring:
1. Enable the SageMaker endpoint to capture data from incoming requests to a trained ML model and the resulting model predictions
2. Create a baseline from the dataset that was used to train the model. The baseline computes metrics and suggests constraints for the metrics. 
3. Create a monitoring schedule specifying what data to collect, how often to collect it, and how to analyze it. Data traffic to your model and predictions from the model are compared to the constraints, and are reported as violations if they are outside the constrained values. You can define multiple monitoring schedule per endpoint
4. Inspect the reports, which compare the latest data with the baseline, and watch for any violations reported and for metrics and notifications from Amazon CloudWatch
5. Implement observability for your ML models with Amazon CloudWatch and event-based architecture with Amazon EventBridge. You can automate data and model updates, model retraining, and user notification based on the data and model quality events

## Section 1 - Setup 

### 1.1 Import required libaries

In [3]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()
sm_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')

# Get Execution role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
RoleArn: arn:aws:iam::006230620263:role/datazone_usr_role_42utgzvtmcm8ls_aehrjybegqbp8w
Region: us-west-2
CPU times

### 1.2 S3 bucket and prefixes

In [41]:
# Setup S3 bucket
bucket = session.default_bucket()
print("S3 Bucket:", bucket)
prefix = "sagemaker/Abalone-ModelQualityMonitor-20201201"
default_bucket_prefix = session.default_bucket_prefix

if default_bucket_prefix:
    prefix = f"{default_bucket_prefix}/{prefix}"

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

S3 Bucket: amazon-sagemaker-006230620263-us-west-2-f717bf909848


In [42]:
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")



Capture path: s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/datacapture
Ground truth path: s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/ground_truth_data/2025-09-23-19-49-33
Report path: s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/reports


##  Section 2 - Discover and Inspect Deployed Endpoint

Let's find the endpoint deployed by our CDK stack and verify its data capture configuration.

### 2. Check deployed Endpoints

In [8]:
def get_deployed_endpoints():
    """Get all InService endpoints with data capture enabled"""
    
    endpoints = []
    
    try:
        response = sm_client.list_endpoints(
            SortBy='CreationTime',
            SortOrder='Descending',
            MaxResults=20
        )
        
        for endpoint in response['Endpoints']:
            if endpoint['EndpointStatus'] == 'InService':
                # Get detailed endpoint info
                endpoint_details = sm_client.describe_endpoint(
                    EndpointName=endpoint['EndpointName']
                )
                
                # Check if data capture is enabled
                data_capture_config = endpoint_details.get('DataCaptureConfig', {})
                
                endpoints.append({
                    'name': endpoint['EndpointName'],
                    'status': endpoint['EndpointStatus'],
                    'creation_time': endpoint['CreationTime'],
                    'data_capture_enabled': data_capture_config.get('EnableCapture', False),
                    'data_capture_status': data_capture_config.get('CaptureStatus', 'Not Configured'),
                    'sampling_percentage': data_capture_config.get('CurrentSamplingPercentage', 0),
                    's3_uri': data_capture_config.get('DestinationS3Uri', 'Not Configured')
                })
        
        return endpoints
        
    except Exception as e:
        print(f"❌ Error listing endpoints: {e}")
        return []

# Get endpoints
endpoints = get_deployed_endpoints()

if endpoints:
    print(f"📊 Found {len(endpoints)} InService endpoint(s):")
    print("-" * 80)
    
    for i, ep in enumerate(endpoints, 1):
        print(f"{i}. {ep['name']}")
        print(f"   Status: {ep['status']}")
        print(f"   Created: {ep['creation_time']}")
        print(f"   Data Capture: {'✅ Enabled' if ep['data_capture_enabled'] else '❌ Disabled'}")
        if ep['data_capture_enabled']:
            print(f"   Capture Status: {ep['data_capture_status']}")
            print(f"   Sampling: {ep['sampling_percentage']}%")
            print(f"   S3 Location: {ep['s3_uri']}")
        print()
else:
    print("❌ No InService endpoints found. Please deploy an endpoint first.")

📊 Found 2 InService endpoint(s):
--------------------------------------------------------------------------------
1. dev-endpoint-20250918-141753
   Status: InService
   Created: 2025-09-18 13:18:26.907000+00:00
   Data Capture: ✅ Enabled
   Capture Status: Started
   Sampling: 100%
   S3 Location: s3://sagemaker-model-monitor-006230620263-us-west-2-dev/data-capture

2. dev-endpoint-20250828-084146
   Status: InService
   Created: 2025-08-28 07:42:39.695000+00:00
   Data Capture: ✅ Enabled
   Capture Status: Started
   Sampling: 100%
   S3 Location: s3://sagemaker-model-monitor-006230620263-us-west-2-dev/data-capture



### 2.2  Verify Captured Data is Available


In [9]:
# Select the endpoint to monitor
if endpoints:
    # Auto-select the first endpoint with data capture enabled
    monitoring_endpoint = None
    for ep in endpoints:
        if ep['data_capture_enabled']:
            monitoring_endpoint = ep
            break
    
    if monitoring_endpoint:
        endpoint_name = monitoring_endpoint['name']
        data_capture_s3_uri = monitoring_endpoint['s3_uri']
        
        print(f"🎯 Selected endpoint for monitoring: {endpoint_name}")
        print(f"📁 Data capture location: {data_capture_s3_uri}")
    else:
        print("❌ No endpoints with data capture enabled found.")
        print("Please ensure your CDK deployment includes data capture configuration.")
        endpoint_name = None
else:
    endpoint_name = None
    print("❌ No endpoints available for monitoring.")

🎯 Selected endpoint for monitoring: dev-endpoint-20250918-141753
📁 Data capture location: s3://sagemaker-model-monitor-006230620263-us-west-2-dev/data-capture


In [11]:


if not endpoint_name:
    print(f"You must have at least on endpoint with data capture configuration enabled!")
else:
    print(f"Checking the data capture configuration for the endpoint {endpoint_name}")
    data_capture_config = sm_client.describe_endpoint(EndpointName=endpoint_name)['DataCaptureConfig']
    data_capture_s3_url = data_capture_config['DestinationS3Uri']
    data_capture_bucket = data_capture_s3_url.split('/')[2]
    data_capture_prefix = '/'.join(data_capture_s3_url.split('/')[3:])

    print(json.dumps(data_capture_config, indent=2))
    print(f"Data capture S3 url: {data_capture_s3_url}")
    
    if not data_capture_config['EnableCapture']:
        print(f"Data capture config for the endpoint {endpoint_name} IS NOT ENABLED. You need to enable data capture for monitoring")

Checking the data capture configuration for the endpoint dev-endpoint-20250918-141753
{
  "EnableCapture": true,
  "CaptureStatus": "Started",
  "CurrentSamplingPercentage": 100,
  "DestinationS3Uri": "s3://sagemaker-model-monitor-006230620263-us-west-2-dev/data-capture"
}
Data capture S3 url: s3://sagemaker-model-monitor-006230620263-us-west-2-dev/data-capture


### 2.3 Generate Test Traffic for Data Capture

Before setting up monitoring, we need to generate some inference traffic to create captured data that we can use for baseline creation.

In [14]:
import pandas as pd
import numpy as np
import os

# Create test_data directory
os.makedirs('test_data', exist_ok=True)
os.makedirs('model', exist_ok=True)

# Create sample abalone data with 10 features 
np.random.seed(42)
n_samples = 500

# Generate synthetic abalone features 
data = {
    'sex_M': np.random.choice([0, 1], n_samples),
    'sex_F': np.random.choice([0, 1], n_samples),
    'length': np.random.uniform(0.075, 0.815, n_samples),
    'diameter': np.random.uniform(0.055, 0.650, n_samples),
    'height': np.random.uniform(0.000, 1.130, n_samples),
    'whole_weight': np.random.uniform(0.002, 2.826, n_samples),
    'shucked_weight': np.random.uniform(0.001, 1.488, n_samples),
    'viscera_weight': np.random.uniform(0.001, 0.760, n_samples),
    'shell_weight': np.random.uniform(0.002, 1.005, n_samples),
    'rings': np.random.randint(1, 30, n_samples)
}

# Create binary age labels (1 if rings > 10, 0 otherwise)
age_labels = (data['rings'] > 10).astype(int)

# Create validation.csv (label first, then features)
df = pd.DataFrame(data)

validation_data = []
for i in range(len(df)):
    row = f"{age_labels[i]}," + ",".join(map(str, df.iloc[i].values))
    validation_data.append(row)

with open('test_data/validation.csv', 'w') as f:
    f.write('\n'.join(validation_data))

# Create test-dataset-input-cols.csv (features only, no labels)
test_data = []
for i in range(len(df)):
    row = ",".join(map(str, df.iloc[i].values))
    test_data.append(row)

with open('test_data/test-dataset-input-cols.csv', 'w') as f:
    f.write('\n'.join(test_data))

# Create upload test file
with open('test_data/upload-test-file.txt', 'w') as f:
    f.write('test file for S3 upload')

print("Sample data files created with 10 features!")

Sample data files created with 10 features!


### 2.4 Prepare S3 layout (baseline, ground truth, reports)

We create/upload paths used by Model Monitor:
- **Baseline data** (optional but recommended): validation set to compute baseline metrics
- **Ground truth stream**: time-partitioned JSONL files for live monitoring
- **Reports**: where the baseline + scheduled jobs write outputs

> You will see paths like:
- Baseline data: `s3://<bucket>/<prefix>/baseline/data/`
- Baseline results: `s3://<bucket>/<prefix>/baseline/results/`
- Ground truth: `s3://<bucket>/<prefix>/ground_truth_data/YYYY/MM/DD/HH/mmss.jsonl`
- Reports: `s3://<bucket>/<prefix>/reports/`


In [15]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer()
)

age_cutoff = 0.5  # Use 0.5 as threshold for binary classification
validate_dataset = "validation_with_predictions.csv"

limit = 50
i = 0
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")
    with open("test_data/validation.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            try:
                probability = float(predictor.predict(input_cols))
                prediction = "1" if probability > age_cutoff else "0"
                baseline_file.write(f"{probability},{prediction},{label}\n")
                i += 1
                if i >= limit:
                    break
                print(".", end="", flush=True)
                sleep(0.5)
            except Exception as e:
                print(f"Error predicting: {e}")
                continue
print()
print("Done!")


.................................................
Done!


In [17]:
# Upload some test files
S3Uploader.upload("test_data/upload-test-file.txt", f"s3://{bucket}/test_upload")
print("Success! You are all set to proceed.")

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Success! You are all set to proceed.


### 2.5  Generate baseline predictions

In [19]:
# Generate baseline predictions
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

baseline_dataset_uri = S3Uploader.upload(f"test_data/{validate_dataset}", baseline_data_uri)
baseline_dataset_uri



Baseline data uri: s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/baselining/data
Baseline results uri: s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/baselining/results
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


's3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/baselining/data/validation_with_predictions.csv'

## Section 3 - Create a baselining job with validation dataset predictions



### 3.1 Create a baselining job with validation dataset predictions


In [20]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

# Create the model quality monitoring object
abalone_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)



sagemaker.config INFO - Applied value from config key = SageMaker.MonitoringSchedule.MonitoringScheduleConfig.MonitoringJobDefinition.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.MonitoringSchedule.MonitoringScheduleConfig.MonitoringJobDefinition.NetworkConfig.VpcConfig.SecurityGroupIds


In [21]:
baseline_job_name = f"smus-AIOps--model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

# Execute the baseline suggestion job
job = abalone_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)

...........................................................!

### 3.2 Review baseline outputs

Open the baseline S3 prefix. You should find:
- `constraints.json` and/or `statistics.json` (schema and reference thresholds)
- `model_quality_metrics.json` (per-class metrics for classification)

If files are missing, check the job logs for parsing issues (e.g., CSV header/column names).


In [22]:
baseline_job = abalone_model_quality_monitor.latest_baselining_job

# View metrics
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,0
confusion_matrix.0.1,17
confusion_matrix.1.0,0
confusion_matrix.1.1,33
recall.value,1.0
recall.standard_deviation,None
precision.value,0.66
precision.standard_deviation,None
accuracy.value,0.66
accuracy.standard_deviation,None


In [23]:
# View constraints
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T


,threshold,comparison_operator
recall,1.0,LessThanThreshold
precision,0.66,LessThanThreshold
accuracy,0.66,LessThanThreshold
true_positive_rate,1.0,LessThanThreshold
true_negative_rate,0.0,LessThanThreshold
false_positive_rate,1.0,GreaterThanThreshold
false_negative_rate,0.0,GreaterThanThreshold
auc,0.557932,LessThanThreshold
f0_5,0.708155,LessThanThreshold
f1,0.795181,LessThanThreshold


### 3.3 Generate prediction data for Model Quality Monitoring

Start generating some artificial traffic. The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as Failed since there is no data to process.

In [24]:
def generate_traffic():
    print("Generating requests...")
    
    with open("test_data/test-dataset-input-cols.csv", "r") as f:
        rows = f.readlines()
    
    for i in range(50):
        try:
            row_index = i % len(rows)
            payload = rows[row_index].rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),
            )["Body"].read()
            print(".", end="", flush=True)
            sleep(0.1)
        except Exception as e:
            print(f"Request {i} failed: {e}")
    
    print(f"\nCompleted! Sent 50 requests.")

generate_traffic()

Generating requests...
..................................................
Completed! Sent 50 requests.


### 3.3 View captured data
Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred.

In [26]:
# Check captured data
import time
time.sleep(30)  # Wait for data capture to process

try:
    # List captured files
    captured_files = S3Downloader.list(f"{data_capture_s3_uri}/{endpoint_name}")
    print(f"Found {len(captured_files)} captured files")
    
    if captured_files:
        # Get latest file
        latest_file = sorted(captured_files)[-1]
        print(f"Latest file: {latest_file.split('/')[-1]}")
        
        # Read and show content
        content = S3Downloader.read_file(latest_file)
        lines = content.strip().split('\n')
        print(f"Records in file: {len(lines)}")
        
        # Show first record
        if lines:
            record = json.loads(lines[0])
            input_data = record['captureData']['endpointInput']['data']
            output_data = record['captureData']['endpointOutput']['data']
            inference_id = record['eventMetadata'].get('inferenceId', 'N/A')
            
            print(f"Sample record:")
            print(f"  Input: {input_data}")
            print(f"  Output: {output_data}")
            print(f"  InferenceId: {inference_id}")
    else:
        print("No captured files found yet")
        
except Exception as e:
    print(f"Error checking captured data: {e}")



sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Found 18 captured files
Latest file: 27-26-569-7aab0f89-fece-473d-b960-e1f1c6554762.jsonl
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Records in file: 50
Sample record:
  Input: 0.0,0.0,0.5916396683746113,0.16515409265897868,0.5865624172039263,0.7410568508701986,1.2315206371720726,0.5115815726045536,0.2893791075739043,10.0
  Output: 13.791704177856445
  InferenceId: 0


### 3.4 Generate synthetic ground truth
Next, start generating ground truth data. The model quality job will fail if there's no ground truth data to merge.

In [27]:
import random

def ground_truth_with_id(inference_id):
    random.seed(inference_id)
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.3 else "0",
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }

def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

NUM_GROUND_TRUTH_RECORDS = 334

def generate_fake_ground_truth_forever():
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        sleep(60 * 60)

gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 334 records to s3://amazon-sagemaker-006230620263-us-west-2-f717bf909848/dzd_51okpx3pb2okls/42utgzvtmcm8ls/dev/sagemaker/Abalone-ModelQualityMonitor-20201201/ground_truth_data/2025-09-23-19-12-49/2025/09/23/19/3135.jsonl
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


### 3.5 Create a monitoring schedule
Now that you have the baseline information and ground truth labels, create a monitoring schedule to run model quality monitoring job

In [32]:
from sagemaker.model_monitor import CronExpressionGenerator


monitor_schedule_name = (
    f"smus-AiOps-ModelQuality-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)
endpointInput = EndpointInput(
    endpoint_name=endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

# Now create the new schedule
response = abalone_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

print(f"New monitoring schedule created")

New monitoring schedule created: None


In [34]:
# You will see the monitoring schedule in the 'Scheduled' status
abalone_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-west-2:006230620263:monitoring-schedule/smus-AiOps-ModelQuality-monitoring-schedule-2025-09-23-1939',
 'MonitoringScheduleName': 'smus-AiOps-ModelQuality-monitoring-schedule-2025-09-23-1939',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2025, 9, 23, 19, 39, 29, 690000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 9, 23, 19, 39, 34, 577000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2025-09-23-19-39-28-855',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'dev-endpoint-20250918-141753',
 'ResponseMetadata': {'RequestId': '8e45c127-cdd3-4ca2-9e01-ca98fef9d6db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8e45c127-cdd3-4ca2-9e01-ca98fef9d6db',
   'strict-transport-security': 'max-age=47304000; includeS

### 3.6 Examine monitoring schedule executions


In [36]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to luanch upto 20 min after the hour.
executions = abalone_model_quality_monitor.list_executions()
executions

[]

In [ ]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = abalone_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

In [ ]:
latest_execution = churn_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

### 3.7 View violations generated by monitoring schedule
If there are any violations compared to the baseline, they will be listed in the reports uploaded to S3.

In [ ]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

## Section 4 - Cleanup

Stop the monitoring schedule and delete any sample endpoint/models you created to avoid charges.  
Also consider emptying the test S3 prefixes if you no longer need the artifacts.


In [ ]:
abalone_model_quality_monitor.delete_monitoring_schedule()
sleep(60)  # actually wait for the deletion

## Troubleshooting

- **"No ground truth found"**: Ensure your JSONL files are uploaded under the expected time partition and the keys (ID/timestamp) align with captured inference records.
- **"Could not parse columns"**: Verify your schema and that `inference_attribute`, `probability_attribute`, and `ground_truth_attribute` match the column names in your files.
- **"No data capture path"**: Re-create the endpoint config with data capture enabled and redeploy, or select a different endpoint that has capture on.
- **Permissions**: The execution role must allow `s3:PutObject`, `s3:GetObject`, `logs:*`, and `sagemaker:*` for jobs/schedules.
